# Linking Counts to Kegg

* Begin with test case


In [ ]:
rctc_1_h = '/home/ndh0004/Documents/keggPthCor/counts/BGI_SRR5341138_accepted_hits.counts'
rctc_1_o = open(rctc_1_h)
rctc_1 = rctc_1_o.read().rstrip('\n').split('\n')


In [2]:
import pickle
f = open('/home/ndh0004/Documents/keggPthCor/gene_dict.pckl','rb')
ab_dict = pickle.load(f)
f.close()
    

In [3]:
for line in rctc_1[0:20]:
    gene, count = line.split('\t')
    if gene in ab_dict:
        print(gene, count, ' '.join(ab_dict[gene]))

PR202_augustus_masked-Super-Scaffold_10-processed-gene-0.444 116 b xo XP_004965949.2 sita:101764737 None
PR202_augustus_masked-Super-Scaffold_10-processed-gene-0.445 0 b xo XP_004965949.2 sita:101764737 None
PR202_augustus_masked-Super-Scaffold_10-processed-gene-0.447 0 b xo XP_004966032.1 sita:101763790 None
PR202_augustus_masked-Super-Scaffold_10-processed-gene-0.451 0 b xo XP_022682880.1 sita:101766674 K01537
PR202_augustus_masked-Super-Scaffold_10-processed-gene-0.453 3 b xo XP_004967201.1 sita:101763801 None
PR202_augustus_masked-Super-Scaffold_10-processed-gene-0.454 0 b xo XP_004966215.1 sita:101768668 K20855


* As above we are just going to create count files with info appended to them
* We will use these for looking at pathways.
* But first let's make this easier with a function.


In [4]:
def create_count(handle,label):
    f = open(handle)
    txt = f.read().rstrip('\n').split('\n')
    f.close()
    handle = handle.split('/')
    out = handle[:-1]
    out.append(label+ '_' + handle[-1])
    out = '/'.join(out)
    fout = open(out, 'w')
    fout.write('gene\tcount\tcall\tconf\tsita\tkSita\tkegg\n')
    for line in txt:
        gene, count = line.split('\t')
        if gene in ab_dict:
            if len(ab_dict[gene]) == 5:
                call, conf, sita, kSita,kegg = ab_dict[gene]
                fout.write('{g}\t{c}\t{ca}\t{co}\t'
                       '{s}\t{si}\t{k}\n'.format(g=gene,
                                                 c=count,
                                                 ca=call,
                                                 co=conf,
                                                 s=sita,
                                                 si=kSita,
                                                 k=kegg))
    fout.close()

In [5]:
create_count('/home/ndh0004/Documents/'
             'keggPthCor/counts/'
             'BGI_SRR5341138_accepted_hits.counts',
             'rctc_1')

quick bash script to fill function 
$p=$( pwd) 
$tail -n +3 BGI_sample_table.csv | awk '{print "create_count(\x27"p"/"$2"\x27,\x27"$1"\x27)"}' p=$p 

In [6]:
create_count('/home/ndh0004/Documents/keggPthCor/counts/BGI_SRR5341139_accepted_hits.counts','lctc_2')
create_count('/home/ndh0004/Documents/keggPthCor/counts/BGI_SRR5341140_accepted_hits.counts','lctc_3')
create_count('/home/ndh0004/Documents/keggPthCor/counts/BGI_SRR5341141_accepted_hits.counts','lctc_4')
create_count('/home/ndh0004/Documents/keggPthCor/counts/BGI_SRR5341142_accepted_hits.counts','sctc_5')
create_count('/home/ndh0004/Documents/keggPthCor/counts/BGI_SRR5341143_accepted_hits.counts','ectc_6')
create_count('/home/ndh0004/Documents/keggPthCor/counts/BGI_SRR5341144_accepted_hits.counts','fctc_7')
create_count('/home/ndh0004/Documents/keggPthCor/counts/BGI_SRR5341145_accepted_hits.counts','lctc_8')
create_count('/home/ndh0004/Documents/keggPthCor/counts/BGI_SRR5341146_accepted_hits.counts','sctc_9')
create_count('/home/ndh0004/Documents/keggPthCor/counts/BGI_SRR5341147_accepted_hits.counts','sctc_10')
create_count('/home/ndh0004/Documents/keggPthCor/counts/BGI_SRR5341148_accepted_hits.counts','sctc_11')

## Agregate the data.
* Again we will begin with a test case. 

In [8]:
oxPhos_h ='/home/ndh0004/code/' \
          'coge_tools/custom_scripts/ref_data/OxPhos.list'
oxPhos_o = open(oxPhos_h)
oxPhos = oxPhos_o.read().rstrip('\n').split('\n')
oxPhos_o.close()


In [9]:
rh = '/home/ndh0004/Documents/' \
     'keggPthCor/counts/' \
     'rctc_1_BGI_SRR5341138_accepted_hits.counts' 
r = open(rh)
dat = r.read().rstrip('\n').split('\n')
print(dat[0:2])
    

['gene\tcount\tcall\tconf\tsita\tkSita\tkegg', 'PR202_augustus_masked-Super-Scaffold_10-processed-gene-0.444\t116\tb\txo\tXP_004965949.2\tsita:101764737\tNone']


In [17]:
ko_dict = {}
for line in dat[1::]:
    gene,count,call,conf,sita,kSita,kegg = line.split('\t')
    count = int(count)
    if kegg in oxPhos:
        if kegg not in ko_dict:
            ko_dict[kegg] = {
                'call_count': {'a':0,'b':0},
                'gene_call_conf_count' : []               
            }
        ko_dict[kegg]['gene_call_conf_count'].append([gene,
                                                     call,
                                                     conf,
                                                     count])
        if call not in  ko_dict[kegg]['call_count']:
            ko_dict[kegg]['call_count'][call] = count
        else:
            ko_dict[kegg]['call_count'][call] += count



In [18]:
key_list = list(ko_dict.keys())
for key in key_list[0:5]:
    print(key, ko_dict[key]['call_count']['a'],\
          ko_dict[key]['call_count']['b'])

K02150 35 0
K02148 0 88
K02113 8 0
K05573 0 111
K02147 354 300


In [19]:
for key in key_list:
    print(key, ko_dict[key]['call_count']['a'],\
          ko_dict[key]['call_count']['b'])

K02150 35 0
K02148 0 88
K02113 8 0
K05573 0 111
K02147 354 300
K03939 54 0
K03952 46 0
K05582 0 15
K03963 0 95
K00413 98 136
K02109 3 0
K02154 358 39
K01507 113 580
K02112 0 0
K00417 105 0
K02133 1317 36
K02266 61 0
K00236 81 0
K11353 0 49
K03943 182 0
K02137 317 171
K03950 0 102
K02155 0 434
K02149 99 41
K03937 1 0
K00235 115 0
K02151 125 0
K00234 25 0
K03661 106 0
K00411 49 392
K02146 0 94
K03955 90 0
K01535 481 1277
K02140 0 0
K02265 169 0
K03942 1144 0
K02267 0 7
K03938 61 27
K11352 0 58


In [ ]:
for key in key_list:
    a = ko_dict[key]['call_count']['a']
    b = ko_dict[key]['call_count']['b']
    if a == 0 and b == 0 :
        print(key, 'gray')
    elif (abs(a - b)/(a + b)) < 0.40:
        print(key,'purple')     
    elif a > b:
        print(key,'blue')
    elif b > a:
        print(key,'red')


K02150 blue
K02148 red
K02113 blue
K05573 red
K02147 purple
K03939 blue
K03952 blue
K05582 red
K03963 red
K00413 purple
K02109 blue
K02154 blue
K01507 red
K02112 gray
K00417 blue
K02133 blue
K02266 blue
K00236 blue
K11353 red
K03943 blue
K02137 purple
K03950 red
K02155 red
K02149 blue
K03937 blue
K00235 blue
K02151 blue
K00234 blue
K03661 blue
K00411 red
K02146 red
K03955 blue
K01535 red
K02140 gray
K02265 blue
K03942 blue
K02267 red
K03938 purple
K11352 red
